In [2]:
#SAC_3 custom env healthy reward 7 ,rewards = healthy_reward - forward_reward

In [3]:
import sys
sys.path.insert(0, "/home/vardan/anaconda3/envs/pytorch/projects/git_humanoid/humanoid/GYM_git")


In [4]:
import gymnasium
print(gymnasium.__file__)


/home/vardan/anaconda3/envs/pytorch/projects/git_humanoid/humanoid/GYM_git/gymnasium/__init__.py


In [5]:
from gymnasium.envs.registration import register

register(
    id="HumanoidV5Custom-v2",
    entry_point="gymnasium.envs.custom_envs.humanoid_v5_custom_v2:HumanoidEnv",
    max_episode_steps=1000,
)


In [6]:
import gymnasium.envs.custom_envs.humanoid_v5_custom  # This triggers the register()
env = gymnasium.make("HumanoidV5Custom-v2", render_mode=None)


In [7]:
obs, _ = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(reward)
    done = terminated or truncated

env.close()


6.916843873452335
6.917174391179784
6.92047443266047
6.911549024978786
6.885356621096631
6.920909789086855
6.904042773057059
6.888851107458818
6.218591410553317
6.817401257611706
6.834237424594611
6.940391723262034
5.667354606174101
6.8272452156738295
6.86491685057686
6.909824254214702
6.90884427586964
6.896548440157048
6.9406559291229915
6.918681592928898
6.8465544093260915
-0.1139195443103108


In [8]:
from stable_baselines3 import SAC, TD3, A2C
import os
import argparse
from stable_baselines3.common.noise import NormalActionNoise
import numpy as np

2025-05-03 22:42:45.627830: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:


# Create base directories
model_base_dir = "models"
log_base_dir = "logs"
os.makedirs(model_base_dir, exist_ok=True)
os.makedirs(log_base_dir, exist_ok=True)

name = "SAC_"

# Find next available index
existing_dirs = [d for d in os.listdir(log_base_dir) if d.startswith(name) and d[4:].isdigit()]
next_index = max([int(d[4:]) for d in existing_dirs], default=-1) + 1

# Create log and model directories with same name
log_dir = os.path.join(log_base_dir, f"{name}{next_index}")
model_dir = os.path.join(model_base_dir, f"{name}{next_index}")
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

def train(model, timesteps):
    iters = 0
    while True:
        iters += 1
        model.learn(total_timesteps=timesteps, reset_num_timesteps=False)
        # Save to models/SAC_X/
        model.save(f"{model_dir}/model_{model.num_timesteps}")
    return model
# def train(model, timesteps, ):

    

#     iters = 0
#     while True:
#         iters += 1
#         model.learn(total_timesteps=timesteps, reset_num_timesteps=False)
#         model.save(f"{model_dir}{name}{next_index}{model.num_timesteps - model.num_timesteps % 1000}")
#     return model





In [10]:
# action_noise = NormalActionNoise(mean=np.zeros(env.action_space.shape), 
#                                  sigma=0.15 * np.ones(env.action_space.shape))
# # model = SAC('MlpPolicy', env, verbose=1, device='cuda', tensorboard_log=log_dir)
# model = SAC(
#     "MlpPolicy",
#     env,
#     verbose=1,
#     device="cuda",
#     tensorboard_log=log_dir,
#     learning_rate=0.0004,        # Default, but can be reduced for more exploration
#     batch_size=512,              # Increase batch size for stability
#     tau=0.004,                   # Lower tau to slow down target updates
#     gamma=0.98,                  # Slightly reduce gamma to value short-term rewards more
#     ent_coef=0.06,               # Encourage more entropy (more randomness)
#     target_update_interval=2,     # Update targets slightly less frequently
#     action_noise=action_noise,     # Inject additional exploration noise
# )

In [11]:
model = SAC.load("models/SAC_2/SACv1_962000", verbose=1, device='cuda', tensorboard_log=log_dir)
model.set_env(env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model = train(model, 20000)

Logging to logs/SAC_5/SAC_0


/home/vardan/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  return F.linear(input, self.weight, self.bias)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 930      |
|    ep_rew_mean     | 5.75e+03 |
| time/              |          |
|    episodes        | 2792     |
|    fps             | 34       |
|    time_elapsed    | 26       |
|    total_timesteps | 963266   |
| train/             |          |
|    actor_loss      | -290     |
|    critic_loss     | 4.44     |
|    ent_coef        | 0.05     |
|    learning_rate   | 0.0003   |
|    n_updates       | 963164   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 902      |
|    ep_rew_mean     | 5.58e+03 |
| time/              |          |
|    episodes        | 2796     |
|    fps             | 34       |
|    time_elapsed    | 46       |
|    total_timesteps | 963919   |
| train/             |          |
|    actor_loss      | -277     |
|    critic_loss     | 11.3     |
|    ent_coef        | 0.05     |
|    learning_